# INITIALISING DB

In [ ]:
def init_connection():
    """Create a database connection that persists across Streamlit reruns"""
    try:
        # Runs locally
        engine = create_engine('sqlite:///database.db') 
        # Test the connection
        with engine.connect() as conn:
            conn.execute(text("SELECT 1"))
        logger.info("Database connection successful")
        return engine
    except Exception as e:
        logger.error(f"Error connecting to database: {e}")
        st.error("Failed to connect to database")
        return None

In [ ]:
def initialize_metadata():
    """Initialize database schema if it doesn't exist"""
    engine = init_connection()
    if not engine:
        return None, None, None, None

    metadata = MetaData()

    # Define tables
    residents = Table(
        'residents', metadata,
        Column('id', Integer, primary_key=True),
        Column('first_name', String),
        Column('last_name', String),
        Column('room_num', Integer)
    )

    care_notes = Table(
        'care_notes', metadata,
        Column('id', Integer, primary_key=True),
        Column('resident_id', Integer, ForeignKey('residents.id')),
        Column('note_text', String),
        Column('staff_name', String),
        Column('sentiment_score', Float),
        Column('timestamp', DateTime)
    )

    try:
        metadata.create_all(engine)
        logger.info("Database schema created successfully")
        return engine, metadata, residents, care_notes
    except Exception as e:
        logger.error(f"Error creating database schema: {e}")
        st.error("Failed to create database schema")
        return None, None, None, None
